This notebook exercises a bug in parameter by:
- Loading a data test
- Create a custom eval funciton (not sure if this is important or not)
- Running a a search with 100 permunations
- Waiting until the job completes by watching CPU usage
- Confirming the kernel is ok
- Running get_status and the kernal dies.

Viewing the logs its looks like all the tasks complete ok.

In [1]:
import graphlab as gl
gl.canvas.set_target('ipynb')
import tools 
import initialise

2016-03-16 21:46:04,603 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.4 started. Logging: /tmp/graphlab_server_1458164763.log


This non-commercial license of GraphLab Create is assigned to kevin.mcisaac@gmail.com and will expire on November 06, 2016. For commercial licensing options, visit https://dato.com/buy/.


## Automatic optimisation

In [2]:
train_data, test_data = initialise.load_data(reload_data=False)
folds = gl.cross_validation.KFold(train_data, 5)

Loading saved processed data
306  features in total


In [3]:
def auc_eval(model, train, test): 
    return {'train_auc':    model.evaluate(train, metric='auc')['auc'],
            'validate_auc': model.evaluate(test,  metric='auc')['auc']}

In [4]:
params = {'target':'TARGET', 'random_seed':8923,
          'class_weights':'auto','metric':'auc', 
          'max_iterations':3,
          'step_size': [0.03],
          'max_depth': [3, 4],
          'column_subsample': [0.8, 0.6]}

job = gl.random_search.create(folds,gl.boosted_trees_classifier.create,
                             params, evaluator=auc_eval, max_models=20)

2016-03-16 21:46:07,916 [INFO] graphlab.deploy.job, 22: Validating job.
2016-03-16 21:46:07,933 [INFO] graphlab.deploy.job, 36: Creating a LocalAsync environment called 'async'.
2016-03-16 21:46:07,938 [INFO] graphlab.deploy.map_job, 186: Validation complete. Job: 'Model-Parameter-Search-Mar-16-2016-21-46-0700000' ready for execution
2016-03-16 21:46:08,064 [INFO] graphlab.deploy.map_job, 192: Job: 'Model-Parameter-Search-Mar-16-2016-21-46-0700000' scheduled.
2016-03-16 21:46:16,223 [INFO] graphlab.deploy.job, 22: Validating job.
2016-03-16 21:46:16,227 [INFO] graphlab.deploy.map_job, 220: A job with name 'Model-Parameter-Search-Mar-16-2016-21-46-0700000' already exists. Renaming the job to 'Model-Parameter-Search-Mar-16-2016-21-46-0700000-0f80a'.
2016-03-16 21:46:16,232 [INFO] graphlab.deploy.map_job, 186: Validation complete. Job: 'Model-Parameter-Search-Mar-16-2016-21-46-0700000-0f80a' ready for execution
2016-03-16 21:46:16,360 [INFO] graphlab.deploy.map_job, 192: Job: 'Model-Param

Wait until all the jobs have finished by wathcing CPU utilisation. Confirm the kernel is still running by executing the line below.

In [8]:
print 'still Ok'

still Ok


Now get the job status. There will be a 10s+ pause then the error message "The kernel appears to have died. It will restart automatically."

In [ ]:
job.get_status()
